In [2]:
import pandas as pd
data = pd.read_csv('Training Data.csv');
print("is data clean :- " , data.notnull().values.all())  # data is clean


input_data1 = data.copy()
input_data1.drop(['CITY','STATE','Risk_Flag'],axis = 1 ,inplace = True)



is data clean :-  True


## For married/single

In [3]:
input_data1['Married/Single'] = input_data1['Married/Single'].apply(lambda x : 1 if  x == 'married' else 0 )
print(input_data1['Married/Single'].value_counts())

0    226272
1     25728
Name: Married/Single, dtype: int64


## For House Ownership

In [4]:
print(input_data1['House_Ownership'].value_counts())
ownership_mapping = {'rented': 0, 'norent_noown': 1, 'owned': 2}
input_data1['House_Ownership'] = input_data1['House_Ownership'].map(ownership_mapping)
print(input_data1['House_Ownership'].value_counts())

rented          231898
owned            12918
norent_noown      7184
Name: House_Ownership, dtype: int64
0    231898
2     12918
1      7184
Name: House_Ownership, dtype: int64


## car_ownership

In [5]:
input_data1['Car_Ownership'].value_counts()
input_data1['Car_Ownership'] = input_data1['Car_Ownership'].apply(lambda x : 1 if  x == 'yes' else 0 )
input_data1['Car_Ownership'].value_counts()

0    176000
1     76000
Name: Car_Ownership, dtype: int64

## Profession

In [11]:
input_data1['Profession'] = input_data1['Profession'].str.replace('_', ' ')

print("number of professions :- ",input_data1['Profession'].nunique())

average_income_by_profession = input_data1.groupby('Profession')['Income'].mean()
# Format the numbers without scientific notation and with commas as thousand separators
average_income_by_profession = average_income_by_profession.apply(lambda x: "{:,.2f}".format(x))

print("average salary of each profession :-\n ",average_income_by_profession)

# Create a mapping for encoding based on average income
profession_encoding = dict(zip(average_income_by_profession.sort_values().index, range(input_data1['Profession'].nunique())))
print(profession_encoding)

# Save the DataFrame to a CSV file this will be used to encoding test data based on training data
df = pd.DataFrame(list(profession_encoding.items()), columns=['Profession', 'Rank'])
df.to_csv('profession_rank.csv', index=False)


# Apply the mapping to create a new column 'Profession_Encoded'
input_data1['Profession_Encoded'] = input_data1['Profession'].map(profession_encoding)
print(input_data1['Profession_Encoded'].value_counts() )


number of professions :-  51
average salary of each profession :-
  Profession
Air traffic controller        4,958,421.71
Analyst                       4,894,181.21
Architect                     5,004,078.75
Army officer                  5,094,472.73
Artist                        5,164,765.18
Aviator                       5,074,103.52
Biomedical Engineer           4,982,527.56
Chartered Accountant          5,002,232.42
Chef                          4,839,915.14
Chemical engineer             5,189,804.99
Civil engineer                4,755,441.02
Civil servant                 4,947,842.64
Comedian                      5,199,538.53
Computer hardware engineer    4,989,531.17
Computer operator             5,112,549.85
Consultant                    5,138,435.81
Dentist                       4,948,818.69
Design Engineer               4,840,220.61
Designer                      5,336,802.23
Drafter                       4,654,804.63
Economist                     4,916,001.92
Engineer          

In [12]:
input_data1_final = input_data1.copy()
input_data1_final.drop(['Profession'],axis = 1,inplace = True)
print(input_data1_final)
output_data1 = data.filter(['Id','Risk_Flag'])
print(output_data1)

            Id   Income  Age  Experience  Married/Single  House_Ownership  \
0            1  1303834   23           3               0                0   
1            2  7574516   40          10               0                0   
2            3  3991815   66           4               1                0   
3            4  6256451   41           2               0                0   
4            5  5768871   47          11               0                0   
...        ...      ...  ...         ...             ...              ...   
251995  251996  8154883   43          13               0                0   
251996  251997  2843572   26          10               0                0   
251997  251998  4522448   46           7               0                0   
251998  251999  6507128   45           0               0                0   
251999  252000  9070230   70          17               0                0   

        Car_Ownership  CURRENT_JOB_YRS  CURRENT_HOUSE_YRS  Profession_Encod

In [13]:
# Remove id from both
train_X = input_data1_final.copy()
train_X.drop(['Id'],axis = 1 , inplace = True )
train_Y = output_data1.copy()
train_Y.drop(['Id'],axis = 1 , inplace = True)
print(train_X.head(10))
print(train_Y.head(10))

    Income  Age  Experience  Married/Single  House_Ownership  Car_Ownership  \
0  1303834   23           3               0                0              0   
1  7574516   40          10               0                0              0   
2  3991815   66           4               1                0              0   
3  6256451   41           2               0                0              1   
4  5768871   47          11               0                0              0   
5  6915937   64           0               0                0              0   
6  3954973   58          14               1                0              0   
7  1706172   33           2               0                0              0   
8  7566849   24          17               0                0              1   
9  8964846   23          12               0                0              0   

   CURRENT_JOB_YRS  CURRENT_HOUSE_YRS  Profession_Encoded  
0                3                 13                  43  
1         

In [14]:
# make array of target variable
Y_train = train_Y['Risk_Flag'].values

In [15]:
# Load test data
x_test = pd.read_csv('test_x.csv')
y_test = pd.read_csv('test_y.csv')

In [16]:
# check if values are pure
print("Is input pure :- ",x_test.notnull().values.all())


Is input pure :-  True


In [17]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100 ,max_samples = 1.0, criterion = 'gini', max_depth = 3,min_samples_split = 10 , min_samples_leaf = 1 )

In [18]:
model.fit(train_X,Y_train)

RandomForestClassifier(max_depth=3, max_samples=1.0, min_samples_split=10)

In [21]:
x_test.head(10)
test_y = y_test['Risk_Flag'].values

In [22]:
print(model.score(x_test,test_y)*100 , "%")

87.16785714285714 %
